# Practical: Atomate 2

This is a practical session on using atomate 2 to run some calculations using the MACE potential that we used last time.


In [ ]:
!pip install atomate2

In [ ]:
from atomate2.ase.jobs import AseRelaxer
from pymatgen.core import Structure, Lattice
from mace.calculators.foundations_models import mace_mp

# Define the structure for silicon
lattice_si = Lattice.from_parameters(a=5.43, b=5.43, c=5.43, alpha=90, beta=90, gamma=90) 

silicon_structure = Structure.from_spacegroup(sg=227, species=["Si"], 
                                              lattice=lattice_si, 
                                              coords=[[0, 0, 0]])

# Create a 2x2x2 supercell
silicon_structure.make_supercell([2, 2, 2])
# Apply some rattling to the structure
silicon_structure.perturb(distance=0.1)

relaxer = AseRelaxer(calculator = mace_mp(model='medium',
                                        default_dtype='float64',
                                        device='cpu'),
                    relax_cell = True,
                    optimizer='LBFGS')

results = relaxer.relax(atoms = silicon_structure,
                        fmax = 0.01,
                        traj_file='silicon_relax.traj',
                        verbose=True)

Using Materials Project MACE for MACECalculator with /Users/zeyudeng/.cache/mace/20231203mace128L1_epoch199model
Using float64 for MACECalculator, which is slower but more accurate. Recommended for geometry optimization.


/Users/zeyudeng/apps/matsci/lib/python3.12/site-packages/mace/calculators/mace.py:139: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(f=model_path, map_location=de

       Step     Time          Energy          fmax
LBFGS:    0 04:24:08     -341.829770        0.136334
LBFGS:    1 04:24:08     -341.830564        0.135427
LBFGS:    2 04:24:09     -341.884174        0.043765
LBFGS:    3 04:24:09     -341.890577        0.001050


In [ ]:
print(f'Elapsed time: {results.elapsed_time:.2f} s')

for i,image in enumerate(results.trajectory):
    print(f'Step{i}: Lattice constant: {image.lattice.abc} Angstrom')


Elapsed time: 2.12 s
Step0: Lattice constant: (10.86, 10.86, 10.86) Angstrom
Step1: Lattice constant: (10.860330493256305, 10.860330493256305, 10.860330493256305) Angstrom
Step2: Lattice constant: (10.894322110283861, 10.89432211028484, 10.89432211028689) Angstrom
Step3: Lattice constant: (10.910589533719515, 10.910589533720819, 10.910589533723915) Angstrom
Step4: Lattice constant: (10.910589533719515, 10.910589533720819, 10.910589533723915) Angstrom


The other way is to use a maker to generate a workflow.

In [86]:
from atomate2.ase.jobs import AseRelaxMaker
from jobflow import run_locally
from ase.calculators.calculator import Calculator

class MACERelaxerMaker(AseRelaxMaker):
    @property
    def calculator(self) -> Calculator:
        """MACE calculator."""
        from mace.calculators.foundations_models import mace_mp
        return mace_mp(model='medium',
                     default_dtype='float64',
                     device='cpu')

workflow = MACERelaxerMaker(name='silicon_relax',
                            relax_cell=True,
                            ionic_step_data=['energy', 'forces', 'stress']).make(silicon_structure)

results = run_locally(workflow)
print(f'Elapsed time: {results} s')




2025-03-26 05:43:59,555 INFO Started executing jobs locally


INFO:jobflow.managers.local:Started executing jobs locally


2025-03-26 05:43:59,563 INFO Starting job - silicon_relax (44e58765-5eb4-4d1f-a984-50ccc5e92c7f)


INFO:jobflow.core.job:Starting job - silicon_relax (44e58765-5eb4-4d1f-a984-50ccc5e92c7f)


Using Materials Project MACE for MACECalculator with /Users/zeyudeng/.cache/mace/20231203mace128L1_epoch199model
Using float64 for MACECalculator, which is slower but more accurate. Recommended for geometry optimization.
Using Materials Project MACE for MACECalculator with /Users/zeyudeng/.cache/mace/20231203mace128L1_epoch199model
Using float64 for MACECalculator, which is slower but more accurate. Recommended for geometry optimization.
Using Materials Project MACE for MACECalculator with /Users/zeyudeng/.cache/mace/20231203mace128L1_epoch199model
Using float64 for MACECalculator, which is slower but more accurate. Recommended for geometry optimization.


...

2025-03-26 05:44:06,274 INFO Finished job - silicon_relax (44e58765-5eb4-4d1f-a984-50ccc5e92c7f)


INFO:jobflow.core.job:Finished job - silicon_relax (44e58765-5eb4-4d1f-a984-50ccc5e92c7f)


2025-03-26 05:44:06,275 INFO Finished executing jobs locally


INFO:jobflow.managers.local:Finished executing jobs locally


Elapsed time: {'44e58765-5eb4-4d1f-a984-50ccc5e92c7f': {1: Response(output=AseStructureTaskDoc(builder_meta=EmmetMeta(emmet_version='0.84.6rc3', pymatgen_version='2025.2.18', run_id=None, batch_id=None, database_version=None, build_date=datetime.datetime(2025, 3, 25, 21, 44, 6, 210406, tzinfo=datetime.timezone.utc), license=None), nsites=64, elements=[Element Si], nelements=1, composition=Composition('Si64'), composition_reduced=Composition('Si1'), formula_pretty='Si', formula_anonymous='A', chemsys='Si', volume=1285.6965820096025, density=2.3215216707306627, density_atomic=20.08900909390004, symmetry=SymmetryData(crystal_system=<CrystalSystem.cubic: 'Cubic'>, symbol='Fd-3m', number=227, point_group='m-3m', symprec=0.1, angle_tolerance=5.0, version='2.5.0'), structure=Structure Summary
Lattice
    abc : 10.873753820678683 10.873753820678534 10.873753820678967
 angles : 90.00000000000017 90.0 90.0
 volume : 1285.6965820096025
      A : 10.873753820678683 -2.2988371867177386e-17 6.265937

In [88]:
from atomate2.forcefields.flows.eos import ForceFieldEosMaker
from atomate2.common.flows.eos import CommonEosMaker    

mace_relax_maker = MACERelaxerMaker(name='silicon_relax',
                                    relax_cell=True,
                                    ionic_step_data=['energy', 'forces', 'stress'])

maker = CommonEosMaker(name='silicon_eos',
                          number_of_frames=5,
                          eos_relax_maker=mace_relax_maker,
                          initial_relax_maker=mace_relax_maker,
                          linear_strain=[-0.05, 0.05])

workflow = maker.make(silicon_structure)

run_locally(workflow)

2025-03-26 05:45:07,892 INFO Started executing jobs locally


INFO:jobflow.managers.local:Started executing jobs locally


2025-03-26 05:45:07,901 INFO Starting job - EOS equilibrium relaxation (dcf31e4b-ce83-4a92-97ba-e606f27c05f1)


INFO:jobflow.core.job:Starting job - EOS equilibrium relaxation (dcf31e4b-ce83-4a92-97ba-e606f27c05f1)


2025-03-26 05:45:07,907 INFO EOS equilibrium relaxation failed with exception:
Traceback (most recent call last):
  File "/Users/zeyudeng/apps/matsci/lib/python3.12/site-packages/jobflow/managers/local.py", line 117, in _run_job
    response = job.run(store=store)
               ^^^^^^^^^^^^^^^^^^^^
  File "/Users/zeyudeng/apps/matsci/lib/python3.12/site-packages/jobflow/core/job.py", line 604, in run
    response = function(*self.function_args, **self.function_kwargs)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: AseRelaxMaker.make() got an unexpected keyword argument 'structure'



INFO:jobflow.managers.local:EOS equilibrium relaxation failed with exception:
Traceback (most recent call last):
  File "/Users/zeyudeng/apps/matsci/lib/python3.12/site-packages/jobflow/managers/local.py", line 117, in _run_job
    response = job.run(store=store)
               ^^^^^^^^^^^^^^^^^^^^
  File "/Users/zeyudeng/apps/matsci/lib/python3.12/site-packages/jobflow/core/job.py", line 604, in run
    response = function(*self.function_args, **self.function_kwargs)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: AseRelaxMaker.make() got an unexpected keyword argument 'structure'



2025-03-26 05:45:07,909 INFO Finished executing jobs locally


INFO:jobflow.managers.local:Finished executing jobs locally


{}

In [ ]:
from atomate2.ase.md import AseMDMaker

class MACEMDMaker(AseMDMaker):
    @property
    def calculator(self) -> Calculator:
        """MACE calculator."""
        from mace.calculators.foundations_models import mace_mp
        return mace_mp(model='medium',
                     default_dtype='float64',
                     device='cpu')

workflow = MACEMDMaker(name='silicon_md',
                        n_steps=100,
                        temperature=800,
                        ensemble='nvt',
                        store_trajectory='si_md.traj',
                        ).make(silicon_structure)

run_locally(workflow, log=str)


2025-03-26 05:50:43,458 INFO Started executing jobs locally


INFO:jobflow.managers.local:Started executing jobs locally


2025-03-26 05:50:43,465 INFO Starting job - silicon_md (f3520e45-c48c-4d4b-82c3-c264be55c3d7)


INFO:jobflow.core.job:Starting job - silicon_md (f3520e45-c48c-4d4b-82c3-c264be55c3d7)


Using Materials Project MACE for MACECalculator with /Users/zeyudeng/.cache/mace/20231203mace128L1_epoch199model
Using float64 for MACECalculator, which is slower but more accurate. Recommended for geometry optimization.
Using Materials Project MACE for MACECalculator with /Users/zeyudeng/.cache/mace/20231203mace128L1_epoch199model
Using float64 for MACECalculator, which is slower but more accurate. Recommended for geometry optimization.
Using Materials Project MACE for MACECalculator with /Users/zeyudeng/.cache/mace/20231203mace128L1_epoch199model
Using float64 for MACECalculator, which is slower but more accurate. Recommended for geometry optimization.


/Users/zeyudeng/apps/matsci/lib/python3.12/site-packages/mace/calculators/mace.py:139: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(f=model_path, map_location=de